# Module 6. Het verband tussen 2 kwantitatieve variabelen (regressie)

Als casus lezen we de [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/) dataset in. Meer info over de dataset vind je op de website, of in de [beschrijving](../data/penguins.md).

In [ ]:
# Importeren van de nodige packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Dataframe
import matplotlib.pyplot as plt                     # Basis visualisatie
from statsmodels.graphics.mosaicplot import mosaic  # Mozaïekdiagram
import seaborn as sns                               # Geavanceerde datavisualisatie
import altair as alt                                # Een alternatief visualisatiesysteem

In [ ]:
# CSV-bestand penguins_size.csv laden
penguins = pd.read_csv('../data/penguins_size.csv')
# penguins.head()
# Categorische variabelen aanduiden
penguins.species = penguins.species.astype('category')
penguins.sex = penguins.sex.astype('category')
penguins.island = penguins.island.astype('category')
# penguins.info()